In [ ]:
# Initial test scrape and API connectivity to GDAX


In [1]:
# Imports

# GDAX Python API: 
    # https://github.com/danpaquin/gdax-python
# GDAX API Documentation: 
    # https://docs.gdax.com
import gdax
    # Import as Git submodule
    # Note: use local install from dev branch to fix mongo connection issue
    
import time
    # Public Endpoint request limit: "3 requests per second, up to 6 requests per second in bursts."
    # https://docs.gdax.com/#rate-limits
    
# Pymongo import for connection to local client DB
from pymongo import MongoClient    

import pandas as pd
import numpy as np
import json


In [2]:
# Establish connection to GDAX public endpoint
public_client = gdax.PublicClient()

In [3]:
# Request time from GDAX server
public_client.get_time()

{'epoch': 1523926441.711, 'iso': '2018-04-17T00:54:01.711Z'}

In [4]:
# Request orderbook at specific channel level for specified product pair
    # Level 2 request = Top 50 bids and asks, aggregated
        # Response: [price, size, number of orders]
    # Level 3 request = Full Orderbook, not aggregated
        # Response: [price, size, order_id]
        # Level 3 response returns the entire orderbook with the associated OrderID; Level 2 does not
public_client.get_product_order_book('BTC-USD', level=2)

{'asks': [['8002', '22.56683859', 42],
  ['8002.11', '0.0015', 1],
  ['8002.12', '0.02', 1],
  ['8002.6', '0.02595954', 1],
  ['8003', '0.001', 1],
  ['8003.06', '0.001', 1],
  ['8003.15', '0.1', 1],
  ['8003.55', '0.00163', 1],
  ['8004', '0.001', 1],
  ['8004.59', '0.95', 1],
  ['8004.6', '1.25173063', 2],
  ['8004.69', '0.955', 1],
  ['8004.88', '0.01555665', 1],
  ['8004.9', '0.86183217', 1],
  ['8005', '5.001', 2],
  ['8006', '0.001', 1],
  ['8006.07', '5.3', 1],
  ['8006.77', '0.8', 1],
  ['8007', '0.051', 2],
  ['8007.49', '0.01457181', 1],
  ['8007.99', '0.01', 1],
  ['8008', '0.001', 1],
  ['8008.12', '2', 1],
  ['8008.71', '0.00435927', 2],
  ['8009', '0.46367948', 2],
  ['8009.04', '0.11384114', 1],
  ['8009.54', '0.0024', 1],
  ['8009.96', '5.2', 1],
  ['8009.99', '1.65284524', 1],
  ['8010', '0.00333043', 2],
  ['8010.48', '0.0313', 1],
  ['8010.96', '0.2', 1],
  ['8011', '0.003504', 2],
  ['8011.01', '1.85', 1],
  ['8011.08', '7.5591', 1],
  ['8011.53', '2', 1],
  ['8012'

In [5]:
#Excessive L3 requests = reduced access/block; Add delay due to response size
time.sleep(4) 
public_client.get_product_order_book('BTC-USD', level=3)

{'asks': [['8002', '2.4757', 'd5e2d8c6-b912-4877-8e19-ca94928b5169'],
  ['8002', '0.1333', '80235e21-8157-43d6-9f9c-98eb6991d218'],
  ['8002', '0.01264809', '5ab89108-8c7e-4a8f-9510-e55a122e0341'],
  ['8002', '0.030242', '728b5bad-8edb-478b-a744-2a2a045f775e'],
  ['8002', '0.6215', '24817d4e-ac60-4747-b85f-5395ff013115'],
  ['8002', '2.1481', 'bdf9d5bf-b3c9-4bb8-ba49-827280b75d5c'],
  ['8002', '0.5', '372fac85-dd25-486c-bcbe-f20dbbe72c50'],
  ['8002', '5.3', '93c9fc23-68b0-4c37-834e-b25046f90efb'],
  ['8002', '0.22', '3e6b3570-1ea5-495c-b041-a54589b015e6'],
  ['8002', '1', 'c62b2c50-ec44-4aec-845d-6033c36185e8'],
  ['8002', '1.773', '2482921c-3796-48a0-af0b-906b1a48981d'],
  ['8002', '2.5', '32ea3706-202c-45e0-a6ef-0966d1607633'],
  ['8002', '0.3521165', '00528e42-bcbd-4680-8480-f97116e17cfe'],
  ['8002', '0.09', 'e3519827-17be-4fb7-8193-d1726d1d4d82'],
  ['8002', '0.1', '4fe848b6-f026-4671-96d5-ed99a53e0602'],
  ['8002', '0.088', '5df8a49e-21e6-406f-9055-7c974f50006e'],
  ['8002', '0.

In [4]:
# GDAX websocket client + mongodb
    # Websocket feed for real time updating of L2 orderbook (and potential order match stream input)
# Mongo database and collection specification:
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client.btcusd_db
btcusd_collection = db.btcusd_collection

In [9]:
# Drop collection if already exists on new scrape:
#db.database_names()
#if btc_usd_collection in db.collection_names() == True:
    #db.['btcusd_db'].drop_collection('btcusd_collection')

In [5]:
# Start instance of Websocketclient
    # Get Level 2 Orderbook for BTC-USD + Match Data

wsClient = gdax.WebsocketClient(url="wss://ws-feed.gdax.com", 
                                products=["BTC-USD"],\
                                message_type="subscribe",\
                                channels =["level2","matches"],\
                                mongo_collection=btcusd_collection,\
                                should_print=False)
time.sleep(4)
request_time_start=public_client.get_time()
wsClient.start()

# Collect 10 minutes of test data and store in mongodb
time.sleep(600)
request_time_end=public_client.get_time()
wsClient.close()

In [11]:
# Second close() to ensure websocket connection termination (potential issue with class method)
wsClient.close()

In [6]:
# Append request times for open/close of websocket stream to dataframe
    # Then save to csv
request_log_df = pd.DataFrame.from_dict({'request start':request_time_start,'request end':request_time_end},orient ='index')
request_log_df.to_csv("test_data/request_log.csv",header=True,encoding='utf-8',index =True)
request_log_df.head()

,iso,epoch
request end,2018-04-17T01:05:03.469Z,1.523927e+09
request start,2018-04-17T00:55:03.354Z,1.523927e+09
